Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils

import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [ ]:
# df = pd.read_pickle(f"{path}output/pickle_results/real_data/real_data_nb_genes.pkl")

In [ ]:
# df = df[~df["nb_genes"].isin(["random_half","all"])]

In [ ]:
df = pd.DataFrame()
dropout = 0.9
lr = 0.4
layers = [200, 40, 60]
# layers = [50]
temperature = 0.07
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())


   
    for nb_genes in ["random_half","all", 500, 1000, 1500, 3000, 5000]:
        for run in range(3):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            
            data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            cluster_number = np.unique(Y).shape[0]
            if nb_genes == "all":
                X = train.preprocess(X, nb_genes=X.shape[1])
            elif nb_genes == "random_half":
                X = train.preprocess(X, nb_genes=X.shape[1])
                X, X_test, Y, y_test = train_test_split(X, Y, train_size=0.5, random_state=run)
            else:
                X = train.preprocess(X, nb_genes=nb_genes)


            dresults = train.run(X,
                                 cluster_number,
                                 dataset,
                                 Y=Y,
                                 nb_epochs=30,
                                 lr=lr,
                                 temperature=temperature,
                                 dropout=dropout,
                                 
                                 layers=layers,
                                 save_to=f"{path}output/real_data/{run}/",
                                 save_pred = False)
            dresults["temperature"] = temperature
            dresults["dropout"] = dropout
            dresults["nb_genes"] = nb_genes
            dresults["layers"] = str(layers)
            dresults["run"] = run
            print(f".", end = "")
            print(f"# {nb_genes}", 
                  dresults.get('kmeans_ari_0',""),
                  dresults.get('kmeans_sil_0', ""),
                  dresults.get('leiden_cal_0', ""), )
            df = df.append(dresults, ignore_index=True)

            df.to_pickle(f"{path}output/pickle_results/real_data/real_data_nb_genes.pkl")

In [ ]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")
sczi = sczi[sczi["dataset"].isin(files)]
sczi.mean()

In [7]:
df = pd.read_pickle(f"../output/pickle_results/real_data/real_data_nb_genes.pkl")
df = df[df["dataset"].isin(files)]
df.groupby(["temperature", "layers", "dropout", "nb_genes"])["kmeans_ari_0"].mean().unstack(["layers", "nb_genes"]).T

temperature                    0.07
dropout                         0.9
layers        nb_genes             
[200, 40, 60] 500          0.736921
              1000         0.701773
              1500         0.680087
              3000         0.655036
              5000         0.627132
              all          0.500469
              random_half  0.495459

In [10]:
df.groupby(["dataset", "nb_genes"])[["kmeans_ari_0"]].mean().unstack("nb_genes")

kmeans_ari_0                                \
nb_genes                              500      1000      1500      3000   
dataset                                                                   
10X_PBMC                         0.701568  0.750348  0.766675  0.781796   
Adam                             0.828620  0.797288  0.781529  0.847966   
Muraro                           0.864083  0.911457  0.705671  0.658466   
Quake_10x_Bladder                0.754433  0.752993  0.753236  0.756035   
Quake_10x_Limb_Muscle            0.986853  0.923673  0.979232  0.897949   
Quake_10x_Spleen                 0.905808  0.503600  0.322492  0.312672   
Quake_Smart-seq2_Diaphragm       0.967334  0.981169  0.982227  0.977997   
Quake_Smart-seq2_Limb_Muscle     0.973210  0.975165  0.976348  0.694734   
Quake_Smart-seq2_Lung            0.706464  0.594065  0.532702  0.484320   
Quake_Smart-seq2_Trachea         0.859771  0.518010  0.490767  0.507872   
Romanov                          0.730948  0.685184  0.668878  0.536790   
Young                            0.595695  0.664553  0.650841  0.656693   
mouse_ES_cell                    0.666633  0.734608  0.742559  0.757898   
mouse_bladder_cell               0.373312  0.443175  0.455703  0.470888   
worm_neuron_cell                 0.139077  0.291311  0.392448  0.483457   

                                                              
nb_genes                          5000       all random_half  
dataset                                                       
10X_PBMC                      0.801425  0.743703    0.670117  
Adam                          0.714970  0.644137    0.568651  
Muraro                        0.634410  0.466427    0.532585  
Quake_10x_Bladder             0.724299  0.461795    0.486576  
Quake_10x_Limb_Muscle         0.780618  0.726908    0.735832  
Quake_10x_Spleen              0.320945  0.285755    0.284052  
Quake_Smart-seq2_Diaphragm    0.974243  0.631156    0.640963  
Quake_Smart-seq2_Limb_Muscle  0.671116  0.491767    0.523573  
Quake_Smart-seq2_Lung         0.424443  0.286191    0.348292  
Quake_Smart-seq2_Trachea      0.518512  0.362228    0.390674  
Romanov                       0.548079  0.451128    0.454097  
Young                         0.631472  0.371944    0.376174  
mouse_ES_cell                 0.710336  0.686670    0.693486  
mouse_bladder_cell            0.460000  0.445522    0.361316  
worm_neuron_cell              0.492115  0.451699    0.365491

In [11]:
optimal = pd.DataFrame(columns = df.columns)

for dataset in df.dataset.unique():
    dff =df[df.dataset == dataset]
    dff = dff.groupby("nb_genes").mean()
    optimalimal_nb_genes = dff[dff["kmeans_sil_0"] == dff["kmeans_sil_0"].max()].index.values[0]
    print(dataset, optimalimal_nb_genes)
    optimal = pd.concat([optimal, df[(df.dataset == dataset) &
        (df.nb_genes == optimalimal_nb_genes)]])
    
optimal.mean()

Quake_Smart-seq2_Trachea 500
Quake_Smart-seq2_Diaphragm 1000
Quake_10x_Spleen 500
Young 1000
mouse_ES_cell 1000
Adam 500
Quake_10x_Bladder 500
Quake_Smart-seq2_Lung 1000
Quake_10x_Limb_Muscle 500
worm_neuron_cell 5000
mouse_bladder_cell 1500
Romanov 500
Quake_Smart-seq2_Limb_Muscle 1000
Muraro 1000
10X_PBMC 500


dropout            0.900000
kmeans_ari_0       0.771789
kmeans_cal_0    4714.539433
kmeans_nmi_0       0.805252
kmeans_sil_0       0.598308
leiden_ari_0       0.347519
leiden_cal_0    3161.166009
leiden_nmi_0       0.636648
leiden_sil_0       0.272854
nb_genes        1066.666667
run                1.000000
temperature        0.070000
time               2.208420
dtype: float64

In [22]:
optimal[["dataset",
         "nb_genes"]].drop_duplicates().reset_index(drop=True).to_pickle(
             f"{path}output/pickle_results/real_data/optimal_input_size.pkl")

In [ ]:
dataset = 'worm_neuron_cell'

In [ ]:
dff.columns

In [ ]:
dff =df[df.dataset == dataset]
dff[[ 'nb_genes', 'kmeans_ari_0','kmeans_nmi_0', 'kmeans_cal_0', 
       'kmeans_sil_0',]]

In [ ]:
opt

In [13]:
category = "real_data"
df_k = optimal[[
    'dataset',
    'dropout',
    'kmeans_ari_0',
    'kmeans_nmi_0',
    'run',
    "time",
#     'kmeans_pred_0',
]].rename(columns={
    'kmeans_ari_0': "ARI",
    'kmeans_nmi_0': "NMI",
#     'kmeans_pred_0': 'pred'
})
df_k["method"] = "Ours+KM"

df_l = optimal[[
    'dataset',
    'dropout',
    'leiden_ari_0',
    'leiden_nmi_0',
    'run',
    'time',
#     'leiden_pred_0',
]].rename(columns={
    'leiden_ari_0': "ARI",
    'leiden_nmi_0': "NMI",
#     'leiden_pred_0': 'pred'
})

df_l["method"] = "Ours+Leiden"

r_data = pd.read_pickle(f"../R/{category}.pkl")

scDeepCluster = pd.read_pickle(f"../output/pickle_results/{category}/{category}_scDeepCluster.pkl")
scDeepCluster["method"] = "scDeepCluster"

sczi = pd.read_pickle(f"../output/pickle_results/{category}/{category}_sczi.pkl")
sczi["method"] = "scziDesk"

scedar = pd.read_pickle(f"../output/pickle_results/{category}/{category}_scedar.pkl")
scedar["method"] = "scedar"

scanpy = pd.read_pickle(f"../output/pickle_results/{category}/{category}_scanpy.pkl")
scanpy["method"] = "scanpy-seurat"

scrna = pd.read_pickle(f"../output/pickle_results/{category}/{category}_scrna.pkl")
scrna["method"] = "scrna"

pca_kmeans = pd.read_pickle(f"../output/pickle_results/{category}/{category}_pca_kmeans.pkl")
pca_kmeans["method"] = "pca_kmeans"

desc = pd.read_pickle(f"../output/pickle_results/{category}/{category}_desc.pkl")
desc["pred"] = desc["pred"].apply(lambda x: x.to_list())
desc["method"] = "desc"

all_data = pd.concat([df_k, df_l, r_data, desc,pca_kmeans, sczi, 
                      scDeepCluster ,scedar, scanpy, scrna])
all_data = all_data.reset_index(drop = True)


In [14]:
all_data = all_data[~all_data["dataset"].str.contains("2100")]

In [15]:
all_data

,dataset,dropout,ARI,NMI,run,time,method,pred,sil
0,Quake_Smart-seq2_Trachea,0.9,0.875742,0.853932,0,0.858098,Ours+KM,NaN,NaN
1,Quake_Smart-seq2_Trachea,0.9,0.823140,0.804126,1,0.848288,Ours+KM,NaN,NaN
2,Quake_Smart-seq2_Trachea,0.9,0.880430,0.861764,2,0.830065,Ours+KM,NaN,NaN
3,Quake_Smart-seq2_Diaphragm,0.9,0.979689,0.957668,0,0.642660,Ours+KM,NaN,NaN
4,Quake_Smart-seq2_Diaphragm,0.9,0.986337,0.971600,1,0.631089,Ours+KM,NaN,NaN
...,...,...,...,...,...,...,...,...,...
570,Muraro,NaN,0.670146,0.727800,1,5.191908,scrna,"[2, 4, 0, 0, 4, 4, 2, 3, 2, 1, 4, 2, 8, 0, 2, ...",0.221733
571,Muraro,NaN,0.670146,0.727800,2,5.178982,scrna,"[2, 4, 0, 0, 4, 4, 2, 3, 2, 1, 4, 2, 8, 0, 2, ...",0.221733
575,10X_PBMC,NaN,0.471505,0.529170,0,8.513393,scrna,"[1, 3, 3, 2, 2, 2, 0, 4, 7, 7, 7, 2, 2, 2, 2, ...",0.068982
576,10X_PBMC,NaN,0.471505,0.529170,1,8.726190,scrna,"[1, 3, 3, 2, 2, 2, 0, 4, 7, 7, 7, 2, 2, 2, 2, ...",0.068982


In [16]:
b = all_data.groupby(["dataset", "method"])["ARI"].mean().unstack("method").round(3).T
b

dataset,10X_PBMC,Adam,Muraro,Quake_10x_Bladder,Quake_10x_Limb_Muscle,Quake_10x_Spleen,Quake_Smart-seq2_Diaphragm,Quake_Smart-seq2_Limb_Muscle,Quake_Smart-seq2_Lung,Quake_Smart-seq2_Trachea,Romanov,Young,mouse_ES_cell,mouse_bladder_cell,worm_neuron_cell
method,,,,,,,,,,,,,,,
Ours+KM,0.702,0.829,0.911,0.754,0.987,0.906,0.981,0.975,0.594,0.860,0.731,0.665,0.735,0.456,0.492
Ours+Leiden,0.406,0.472,0.366,0.178,0.434,0.093,0.339,0.376,0.360,0.178,0.275,0.521,0.337,0.434,0.443
cidr,0.528,0.051,0.527,0.734,0.677,0.648,0.969,0.781,0.316,0.539,0.322,0.266,0.731,0.214,0.035
desc,0.569,0.556,0.551,0.222,0.466,0.192,0.557,0.392,0.467,0.189,0.280,0.510,0.698,0.643,0.355
pca_kmeans,0.190,0.022,0.357,0.528,0.419,0.066,0.107,0.035,0.112,-0.003,0.105,0.214,0.292,0.169,0.047
raceid,0.425,0.021,0.827,0.732,0.540,0.405,0.126,0.501,0.172,0.492,0.417,0.438,0.645,0.343,NaN
scDeepCluster,0.760,0.321,0.388,0.606,0.737,0.408,0.845,0.583,0.553,0.572,0.536,0.304,0.704,0.538,0.521
scanpy-seurat,0.558,0.512,0.446,0.227,0.427,0.153,0.492,0.391,0.340,0.153,0.336,0.473,0.776,0.639,0.405
scedar,0.177,0.133,0.314,0.128,0.220,0.021,0.368,0.404,0.242,0.218,0.138,0.131,0.252,0.302,0.251


In [17]:
b.rank(ascending=False, method = "min")

dataset,10X_PBMC,Adam,Muraro,Quake_10x_Bladder,Quake_10x_Limb_Muscle,Quake_10x_Spleen,Quake_Smart-seq2_Diaphragm,Quake_Smart-seq2_Limb_Muscle,Quake_Smart-seq2_Lung,Quake_Smart-seq2_Trachea,Romanov,Young,mouse_ES_cell,mouse_bladder_cell,worm_neuron_cell
method,,,,,,,,,,,,,,,
Ours+KM,2.0,2.0,1.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,3.0,5.0,2.0
Ours+Leiden,10.0,6.0,11.0,12.0,10.0,11.0,10.0,10.0,5.0,10.0,10.0,3.0,10.0,6.0,3.0
cidr,6.0,10.0,8.0,4.0,5.0,4.0,2.0,4.0,7.0,4.0,7.0,9.0,4.0,12.0,12.0
desc,4.0,3.0,7.0,11.0,9.0,9.0,5.0,8.0,4.0,9.0,9.0,4.0,6.0,1.0,5.0
pca_kmeans,12.0,12.0,12.0,9.0,12.0,12.0,12.0,13.0,13.0,12.0,13.0,12.0,12.0,13.0,11.0
raceid,9.0,13.0,2.0,5.0,6.0,6.0,11.0,6.0,10.0,6.0,4.0,7.0,8.0,9.0,NaN
scDeepCluster,1.0,7.0,10.0,8.0,4.0,5.0,4.0,5.0,3.0,3.0,3.0,8.0,5.0,3.0,1.0
scanpy-seurat,5.0,4.0,9.0,10.0,11.0,10.0,7.0,9.0,6.0,11.0,5.0,6.0,2.0,2.0,4.0
scedar,13.0,9.0,13.0,13.0,13.0,13.0,9.0,7.0,8.0,8.0,12.0,13.0,13.0,10.0,6.0


In [ ]:
xx =all_data.groupby(["dataset", "method"])["ARI"].mean().unstack("method").round(3).reset_index()
xx =all_data.groupby(["dataset", "method"])["ARI"].mean().round(3).reset_index()
xx

In [ ]:
xx.groupby(["dataset"])["ARI"].rank(ascending = True, method = 'first')

In [ ]:
xx.groupby(["dataset", "method"])["ARI"].rank("dense", ascending=False)

In [ ]:
# r = df[(df["layers"]=="[40]")
#        &(df["temperature"]==0.01)
#        &(df["lr"]==1e-5)
#        &(df["dropout"]==0.9)
#       ]
r = df[(df["layers"]=="[200, 100, 30, 30]")
       &(df["temperature"]==0.07)
       &(df["lr"]==1e-5)
       &(df["dropout"]==0.9)
      ]


r.mean()

In [ ]:
comb = pd.merge(r, sczi, on=["dataset", "run"])[[
    "dataset", "kmeans_ari_0", "ARI", "kmeans_nmi_0", "NMI"
]].rename(columns = {"kmeans_ari_0": "contrative-sc", "ARI": "sczi"})
comb

In [ ]:
comb =pd.melt(comb, id_vars=["dataset"], value_vars=["contrative-sc", "sczi"])

In [ ]:
comb.head()